In [135]:
%%script bash
apt install libeigen3-dev
ln -sf /usr/include/eigen3/Eigen /usr/include/Eigen

Reading package lists...
Building dependency tree...
Reading state information...
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [136]:
%%script bash
sudo rm -r meshes
mkdir meshes
cd meshes
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/base.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/forearm.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/upperarm.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/shoulder.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/wrist1.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/wrist2.stl
wget https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/wrist3.stl
cd ..

--2024-03-17 00:09:31--  https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/base.stl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28984 (28K) [application/octet-stream]
Saving to: ‘base.stl’

     0K .......... .......... ........                        100% 10.8M=0.003s

2024-03-17 00:09:31 (10.8 MB/s) - ‘base.stl’ saved [28984/28984]

--2024-03-17 00:09:31--  https://raw.githubusercontent.com/PeterJochem/CUDA_Robotics_Kernels/main/meshes/UR5/collision/forearm.stl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting

In [137]:
%%script bash
ls ./meshes

base.stl
forearm.stl
shoulder.stl
upperarm.stl
wrist1.stl
wrist2.stl
wrist3.stl


In [138]:
%%writefile vectors.hpp
#pragma once
#include <iostream>
#include <Eigen/Dense>

using Eigen6Vector = Eigen::Matrix<float, 6, 1>;
using Eigen6by6Matrix = Eigen::Matrix<float, 6, 6>;
using Eigen3by3Matrix = Eigen::Matrix<float, 3, 3>;

class Vector3 {

  public:
    __device__ Vector3();
    Vector3(float, float, float);
    float x;
    float y;
    float z;
    __device__ Vector3 operator-(const Vector3&) const;
    __device__ Vector3 operator-(Vector3&) const;
    __device__ float operator*(Vector3&) const;
    __device__ float operator*(const Vector3&) const;
    __device__ void normalize();
};

class Vector6 {

  public:
    __device__ Vector6(void);
    __device__ Vector6(float, float, float, float, float, float);
    __device__ Eigen6Vector to_eigen(void);
    float x, y, z, p, q, w;
};

Overwriting vectors.hpp


In [139]:
%%writefile vectors.cu
#include "vectors.hpp"

__device__ void Vector3::normalize() {

    float magnitude = (x * x) + (y * y) + (z * z);
    x = x / magnitude;
    y = y / magnitude;
    z = z / magnitude;
}

__device__ Vector3::Vector3(): x(0.), y(0.), z(0.) {
}

Vector3::Vector3(float x, float y, float z): x(x), y(y), z(z) {
}

__device__ Vector3 Vector3::operator-(const Vector3& right) const {

    Vector3 v;
    v.x = x - right.x;
    v.y = y - right.y;
    v.z = z - right.z;
    return v;
}

__device__ Vector3 Vector3::operator-(Vector3& right) const {

    Vector3 v;
    v.x = x - right.x;
    v.y = y - right.y;
    v.z = z - right.z;
    return v;
}

__device__ float Vector3::operator*(Vector3& right) const {

    return (x * right.x) + (y * right.y) + (z * right.z);
}

__device__ float Vector3::operator*(const Vector3& right) const {

    return (x * right.x) + (y * right.y) + (z * right.z);
}


__device__ Vector6::Vector6(void): x(0.), y(0.), z(0.), p(0.), q(0.), w(0.) {

}

__device__ Vector6::Vector6(float x, float y, float z, float p, float q, float w): x(x), y(y), z(z), p(p), q(q), w(w) {

}

__device__ Eigen6Vector Vector6::to_eigen(void) {

  Eigen6Vector vector;
  vector << x, y, z, p, q, w;
  return vector;
}

Overwriting vectors.cu


In [140]:
%%writefile screw.hpp
#pragma once
#include "vectors.hpp"

class Screw {

  public:
    Screw();
    Screw(Vector3, Vector3);
    //Screw(float*);

    Vector3 w;
    Vector3 v;
};

Overwriting screw.hpp


In [141]:
%%writefile screw.cu
#include "screw.hpp"
Screw::Screw(): w(Vector3(0., 0., 0.)), v(Vector3(0., 0., 0.)) {
}

Screw::Screw(Vector3 w, Vector3 v): w(w), v(v) {
}

Overwriting screw.cu


In [168]:
%%writefile transformation.hpp
#pragma once
#include "screw.hpp"
#include "vectors.hpp"
#include <iostream>
#include <Eigen/Dense>


inline __device__ float dot_product(float* row, float* col, int size) {

  float result = 0.;
  for (int i = 0; i < size; i++) {
    result += (row[i] * col[i]);
  }

  return result;
}

class Transformation {

  public:
    static const int num_entries_in_matrix = 16;
    float values[num_entries_in_matrix];

    __device__ Transformation();
    __device__ ~Transformation();
    __device__ Transformation(float*);
    __device__ Transformation operator+(const Transformation&);
    __device__ Transformation operator*(const Transformation&);
    __device__ Transformation operator*(const float);
    __device__ Transformation bracket_w(const Vector3& w);
    __device__ Transformation bracket_w_squared(const Vector3& w);
    __device__ Vector3 operator*(const Vector3&) const;
    __device__ Vector3 operator*(Vector3&) const;
    __device__ void set_translation(const Vector3&);
    __device__ float* get_row(int) const;
    __device__ float* get_column(int) const;
    __device__ Vector3 euler_angles(void) const;
    __device__ Transformation& operator=(const Transformation&);
    __device__ Transformation(Transformation&);
    __device__ Vector6 pose(void) const;
    __device__ Vector3 translatation(void) const;
    //void print() const;
};

Overwriting transformation.hpp


In [175]:
%%writefile transformation.cu
#include "transformation.hpp"


__device__ Vector3 Transformation::translatation(void) const {

  Vector3 result;

  result.x = values[3];
  result.y = values[7];
  result.z = values[11];

  return result;
}

__device__ Vector6 Transformation::pose(void) const {
  float x, y, z;

  x = values[3];
  y = values[7];
  z = values[11];

  Vector3 angles = euler_angles();

  return Vector6(x, y, z, angles.x, angles.y, angles.z);
  //return Vector6(x, y, z, x, y, z);
}

__device__ Transformation::Transformation(Transformation& right) {

  for (int i = 0; i < num_entries_in_matrix; i++) {
      values[i] = right.values[i];
  }
}

 __device__ Transformation& Transformation::operator=(const Transformation& right) {

    for (int i = 0; i < num_entries_in_matrix; i++) {
      values[i] = right.values[i];
    }

    return *this;
  }

__device__ Vector3 Transformation::euler_angles(void) const {
  // https://lavalle.pl/planning/node103.html

  Vector3 result;

  float r11 = values[0];
  float r21 = values[4];
  float r31 = values[8];
  float r32 = values[9];
  float r33 = values[10];

  result.x = atan2f(r21, r11); // atan2f(r11, r21);
  result.y = atan2f(-r31, sqrtf((r32 * r32) + (r33 * r33))); // atan2f(sqrtf((r32 * r32) + (r33 * r33)), -r31);
  result.z = atan2f(r32, r33); // atan2f(r33, r32);

  return result;
}

__device__ Transformation::Transformation() {

  // Create the identity matrix.
  for (int i = 0; i < num_entries_in_matrix; i++) {
    values[i] = 0.0;
  }

  values[0] = 1.;
  values[5] = 1.;
  values[10] = 1.;
  values[15] = 1.;
}

__device__ Transformation::~Transformation() {
  //free(values);
}

__device__ Transformation::Transformation(float* input_values) {

  for (int i = 0; i < num_entries_in_matrix; i++) {
    this->values[i] = input_values[i];
  }
}

__device__ Transformation Transformation::operator+(const Transformation &right) {

  Transformation result = Transformation();

  for (int i = 0; i < num_entries_in_matrix; i++) {
    result.values[i] = values[i] + right.values[i];
  }

  result.values[num_entries_in_matrix - 1] = 1.0;
  return result;
}

__device__ Transformation Transformation::operator*(const Transformation &right) {

  Transformation result = Transformation();
  int num_entries_in_row = 4;

  for (int row_index = 0; row_index < num_entries_in_row; row_index++) {
    float* row = get_row(row_index);

    for (int col_index = 0; col_index < num_entries_in_row; col_index++) {
      float* column = right.get_column(col_index);
      result.values[(row_index * 4) + col_index] = dot_product(row, column, num_entries_in_row);
      free(column);
    }
    free(row);
  }

  return result;
}

__device__ Transformation Transformation::operator*(const float c) {

  Transformation result = Transformation();

  for (int i = 0; i < num_entries_in_matrix - 1; i++) {
    result.values[i] = values[i] * c;
  }

  return result;
}

__device__ Transformation Transformation::bracket_w(const Vector3& w) {

  Transformation result = Transformation();

  // First row.
  result.values[0] = 0.;
  result.values[1] = -w.z;
  result.values[2] = w.y;

  // Second row.
  result.values[4] = w.z;
  result.values[5] = 0;
  result.values[6] = -w.x;

  // Third row.
  result.values[8] = -w.y;
  result.values[9] = w.x;
  result.values[10] = 0.;

  return result;
}

__device__ Transformation Transformation::bracket_w_squared(const Vector3& w) {

  Transformation result = Transformation();

  // First row.
  result.values[0] = -(w.z * w.z) - (w.y * w.y);
  result.values[1] = w.y * w.x;
  result.values[2] = w.z * w.x;

  // Second row.
  result.values[4] = w.y * w.x;
  result.values[5] = -(w.z * w.z) - (w.x * w.x);
  result.values[6] = w.z * w.y;

  // Third row.
  result.values[8] = w.z * w.x;
  result.values[9] = w.z * w.y;
  result.values[10] = -(w.y * w.y) - (w.x * w.x);

  return result;
}

__device__ Vector3 Transformation::operator*(const Vector3& right) const {

  Vector3 result = Vector3();
  result.x = (values[0] * right.x) + (values[1] * right.y) + (values[2] * right.z) + values[3];
  result.y = (values[4] * right.x) + (values[5] * right.y) + (values[6] * right.z) + values[7];
  result.z = (values[8] * right.x) + (values[9] * right.y) + (values[10] * right.z) + values[11];
  return result;
}

__device__ Vector3 Transformation::operator*(Vector3& right) const {

  Vector3 result = Vector3();
  result.x = (values[0] * right.x) + (values[1] * right.y) + (values[2] * right.z) + values[3];
  result.y = (values[4] * right.x) + (values[5] * right.y) + (values[6] * right.z) + values[7];
  result.z = (values[8] * right.x) + (values[9] * right.y) + (values[10] * right.z) + values[11];
  return result;
}

__device__ void Transformation::set_translation(const Vector3& v) {
  values[3] = v.x;
  values[7] = v.y;
  values[11] = v.z;
}

__device__ float* Transformation::get_row(int index) const {

  float* row = ((float*)values) + (index * 4);
  float* copy = (float*) malloc(sizeof(float) * 4);
  for (int i = 0; i < 4; i++) {
    copy[i] = row[i];
  }

  return copy;
}

__device__ float* Transformation::get_column(int index) const {

  float* column = (float*) malloc(sizeof(float) * 4);
  for (int i = 0; i < 4; i++) {
    column[i] = values[index + (4 * i)];
  }

  return column;
}

//void Transformation::print() const;

Overwriting transformation.cu


In [144]:
%%writefile core.hpp
#pragma once
#include "transformation.hpp"
#include "vectors.hpp"
#include "screw.hpp"
#include <iostream>
#include <Eigen/Dense>

using Eigen6Vector = Eigen::Matrix<float, 6, 1>;
using Eigen6by6Matrix = Eigen::Matrix<float, 6, 6>;
using Eigen3by3Matrix = Eigen::Matrix<float, 3, 3>;

__device__ Transformation exponential_rotation(Vector3& w, float theta);
__device__ Vector3 rigid_body_exponential_translation(Vector3& w, Vector3& v, float theta);
__device__ Transformation multiply(Transformation* transformations, int n);
__device__ Transformation rigid_body_exponential(Screw& screw, float theta);
__device__ Transformation forward_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints);
__device__ Transformation* forward_kinematics_joint_transformations(Screw* screws, float* thetas, int num_joints);
__device__ Eigen6by6Matrix jacobian(Screw* screws, float* thetas, Transformation& home_position, int num_joints);
__device__ Eigen3by3Matrix inverse(Eigen3by3Matrix& matrix);
__device__ Eigen6by6Matrix inverse(Eigen6by6Matrix& matrix);
Eigen6by6Matrix inverse_cpu(Eigen6by6Matrix& matrix);
__device__ float absolute_error(Eigen6Vector& left, Eigen6Vector& right);
__device__ Eigen6Vector inverse_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints);
__device__ Transformation test_inverse(Transformation* result);
__global__ void test_inverse_kernel(Transformation* result);
__global__ void forward_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Transformation* result_transformations);
__global__ void inverse_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Eigen6Vector* results);
void create_joint_angles(float a, float b, float c, float d, float e, float f, float* output);

Overwriting core.hpp


In [145]:
%%writefile core.cu
#include "core.hpp"


__device__ Transformation exponential_rotation(Vector3& w, float theta) {
  /* See page 113 of Modern Robotics.
  * I + sin θ[ωˆ] + (1 − cos θ)[ωˆ]2
  */

  Transformation identity = Transformation();
  Transformation sin_theta_w = Transformation().bracket_w(w) * sin(theta);
  Transformation cos_theta_w_squared = Transformation().bracket_w_squared(w) * (1.0 - cos(theta));

  return identity + sin_theta_w + cos_theta_w_squared;
}

__device__ Vector3 rigid_body_exponential_translation(Vector3& w, Vector3& v, float theta) {
  /* See page 113 of Modern Robotics.
   * (Iθ + (1 − cos θ)[ω] + (θ − sin θ)[ω]2)v
   * (A + B + C) * (v)
   * (3 x 3 matrix) * (3 x 1 matrix) = (3 x 1 matrix)
   */

  Transformation identity_theta = Transformation() * theta;
  Transformation sin_theta_w = Transformation().bracket_w(w) * (1.0 - cos(theta));
  Transformation cos_theta_w_squared = Transformation().bracket_w_squared(w) * (theta - sin(theta));

  return (identity_theta + sin_theta_w + cos_theta_w_squared) * v;
}

__device__ Transformation multiply(Transformation* transformations, int n) {

  // Multiply each transformation together.
  Transformation result = Transformation();
  for (int i = 0; i < n; i++) {
    result = result * transformations[i];
  }

  return result;
}

__device__ Transformation rigid_body_exponential(Screw& screw, float theta) {

  // Create the transformations.
  Transformation result = exponential_rotation(screw.w, theta);
  Vector3 translatation = rigid_body_exponential_translation(screw.w, screw.v, theta);

  result.set_translation(translatation);
  return result;
}

__device__ Transformation forward_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  int num_transformations = num_joints + 1;
  Transformation* transformations = (Transformation*) malloc(sizeof(Transformation) * num_transformations);
  for (int i = 0; i < num_joints; i++) {
    screws[i].w.normalize();
    transformations[i] = rigid_body_exponential(screws[i], thetas[i]);
  }
  transformations[num_joints] = home_position;
  return multiply(transformations, num_transformations);
}

__device__ Transformation* forward_kinematics_joint_transformations(Screw* screws, float* thetas, int num_joints) {

  Transformation* transformations = (Transformation*) malloc(sizeof(Transformation) * num_joints);
  Transformation* transformation_results = (Transformation*) malloc(sizeof(Transformation) * num_joints);
  for (int i = 0; i < num_joints; i++) {
    screws[i].w.normalize();
    transformations[i] = rigid_body_exponential(screws[i], thetas[i]);
  }

  for (int i = 0; i < num_joints; i++) {
    transformation_results[i] = multiply(transformations, i + 1);
  }

  free(transformations);
  return transformation_results;
}

__device__ Eigen6by6Matrix jacobian(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  Transformation* transformations = (Transformation*) malloc(sizeof(Transformation) * (num_joints + 1));
  Eigen6by6Matrix jacobian_matrix(num_joints, 6);
  for (int i = 0; i < num_joints; i++) {
    screws[i].w.normalize();
    transformations[i] = rigid_body_exponential(screws[i], thetas[i]);
  }
  transformations[num_joints] = home_position;

  Transformation result_original = multiply(transformations, num_joints + 1);
  Eigen6Vector pose_original = result_original.pose().to_eigen();

  // Move into parameterized struct.
  float delta = 0.0001;

  // Multiply each transformation together.
  for (int i = 0; i < num_joints; i++) {

    Transformation original = transformations[i];
    Transformation altered_transformation = rigid_body_exponential(screws[i], thetas[i] + delta);
    transformations[i] = altered_transformation;

    Transformation result = multiply(transformations, num_joints + 1);

    // Add a column of the jacobian matrix.
    Eigen6Vector column = result.pose().to_eigen() - pose_original;
    jacobian_matrix.col(i) = column / delta;

    transformations[i] = original;
  }


  //free(transformations);
  //return &result;
  return jacobian_matrix;
}

__device__ Eigen3by3Matrix inverse(Eigen3by3Matrix& matrix) {

  float a, b, c, d, e, f, g, h, i;
  float* values = matrix.data(); // Column major.
  a = values[0];
  d = values[1];
  g = values[2];

  b = values[3];
  e = values[4];
  h = values[5];

  c = values[6];
  f = values[7];
  i = values[8];

  float epsilon = 0.00001;
  float divisor = (a * (e * i - f * h)) + (-b * (d * i - f * g)) + (c * (d * h - e * g));
  if (fabsf(divisor) < epsilon) {
    divisor += epsilon;
  }

  Eigen3by3Matrix result;
  result(0, 0) = e * i - f * h;
  result(0, 1) = c * h - b * i;
  result(0, 2) = b * f - c * e;

  result(1, 0) = f * g - d * i;
  result(1, 1) = a * i - c * g;
  result(1, 2) = c * d - a * f;

  result(2, 0) = d * h - e * g;
  result(2, 1) = b * g - a * h;
  result(2, 2) = a * e - b * d;

  result = result / divisor;

  return result;
}

__device__ Eigen6by6Matrix inverse(Eigen6by6Matrix& matrix) {
  // Eigen only supports GPU inverse up to matrices of size 4x4.
  // See https://en.wikipedia.org/wiki/Block_matrix
  using namespace Eigen;

  Eigen3by3Matrix A = matrix.topLeftCorner(3, 3);
  Eigen3by3Matrix B = matrix.topRightCorner(3, 3);
  Eigen3by3Matrix C = matrix.bottomLeftCorner(3, 3);
  Eigen3by3Matrix D = matrix.bottomRightCorner(3, 3);

  Eigen3by3Matrix A_i = inverse(A);
  Eigen3by3Matrix B_i = inverse(B);
  Eigen3by3Matrix C_i = inverse(C);
  Eigen3by3Matrix D_i = inverse(D);

  Eigen3by3Matrix A_i_B = A_i * B;
  Eigen3by3Matrix C_A_i_B = C * A_i_B;
  Eigen3by3Matrix C_A_i = C * A_i;
  Eigen3by3Matrix aaa = D - C_A_i_B;
  Eigen3by3Matrix D_minus_C_A_i_B_inverse = inverse(aaa);

  Eigen3by3Matrix A_result = A_i + (A_i_B * D_minus_C_A_i_B_inverse) * C_A_i;

  Eigen3by3Matrix B_result = (-1 * A_i_B) * D_minus_C_A_i_B_inverse;

  Eigen3by3Matrix C_result = -1 * D_minus_C_A_i_B_inverse * C_A_i;

  Eigen3by3Matrix D_result = D_minus_C_A_i_B_inverse;

  Eigen6by6Matrix result;
  result.topLeftCorner<3, 3>() = A_result;
  result.topRightCorner<3, 3>() = B_result;
  result.bottomLeftCorner<3, 3>() = C_result;
  result.bottomRightCorner<3, 3>() = D_result;

  return result;
}

Eigen6by6Matrix inverse_cpu(Eigen6by6Matrix& matrix) {
  // Eigen only supports GPU inverse up to matrices of size 4x4.
  // See https://en.wikipedia.org/wiki/Block_matrix
  using namespace Eigen;

  Eigen3by3Matrix A = matrix.topLeftCorner(3, 3);
  Eigen3by3Matrix B = matrix.topRightCorner(3, 3);
  Eigen3by3Matrix C = matrix.bottomLeftCorner(3, 3);
  Eigen3by3Matrix D = matrix.bottomRightCorner(3, 3);

  Eigen3by3Matrix A_i = A.inverse();
  Eigen3by3Matrix B_i = B.inverse();
  Eigen3by3Matrix C_i = C.inverse();
  Eigen3by3Matrix D_i = D.inverse();

  Eigen3by3Matrix A_i_B = A_i * B;
  Eigen3by3Matrix C_A_i_B = C * A_i_B;
  Eigen3by3Matrix C_A_i = C * A_i;
  Eigen3by3Matrix D_minus_C_A_i_B_inverse = (D - C_A_i_B).inverse();

  Eigen3by3Matrix A_result = A_i + (A_i_B * D_minus_C_A_i_B_inverse) * C_A_i;

  Eigen3by3Matrix B_result = (-1 * A_i_B) * D_minus_C_A_i_B_inverse;

  Eigen3by3Matrix C_result = -1 * D_minus_C_A_i_B_inverse * C_A_i;

  Eigen3by3Matrix D_result = D_minus_C_A_i_B_inverse;

  Eigen6by6Matrix result;
  result.topLeftCorner<3, 3>() = A_result;
  result.topRightCorner<3, 3>() = B_result;
  result.bottomLeftCorner<3, 3>() = C_result;
  result.bottomRightCorner<3, 3>() = D_result;

  return result;
}

__device__ float absolute_error(Eigen6Vector& left, Eigen6Vector& right) {

  return (left - right).norm();
}

__device__ Eigen6Vector inverse_kinematics(Screw* screws, float* thetas, Transformation& home_position, int num_joints) {

  // Move into configurable struct.
  int max_num_iterations = 200;
  int max_allowed_error = 0.000001;
  float epsilon = 0.25;

  Eigen6Vector desired_pose;
  desired_pose << 0.0957158, 0.109065 + 0.12, 0.987924, 0., 0., 3.14/2.0;
  Eigen6Vector current_pose = forward_kinematics(screws, thetas, home_position, num_joints).pose().to_eigen();

  int i = 0;
  while (i < max_num_iterations && absolute_error(current_pose, desired_pose) > max_allowed_error) {

    // Calculate how to update thetas.
    current_pose = forward_kinematics(screws, thetas, home_position, num_joints).pose().to_eigen();
    Eigen6by6Matrix jacobian_matrix = jacobian(screws, thetas, home_position, num_joints);
    Eigen6by6Matrix jacobian_matrix_inverse = inverse(jacobian_matrix);
    Eigen::VectorXf theta_deltas = jacobian_matrix_inverse * (desired_pose - current_pose);

    // Apply the theta updates.
    for (int j = 0; j < num_joints; j++) {
      float update = theta_deltas[j] * epsilon / (theta_deltas.norm() + 0.000001);
      thetas[j] += update;
    }

    i = i + 1;
  }


  Eigen6Vector result_thetas;
  result_thetas << thetas[0], thetas[1], thetas[2], thetas[3], thetas[4], thetas[5];
  //return result_thetas;
  return current_pose;
}

__device__ Transformation test_inverse(Transformation* result) {

  Transformation identity = Transformation();

  Vector3 t;
  t.x = 1;
  t.y = 2;
  t.z = 3;
  identity.set_translation(t);

  float* values = (float*) malloc(sizeof(float) * 6 * 6);
  for (int i = 0; i < 6; i++) {
    for (int j = 0; j < 6; j++) {
      values[(i * 6) + j] = 0.0;
      if (i == j) {
        values[(i * 6) + j] = 1.0;
      }
    }
  }

  //values[0] = 10.0;
  //values[6] = 4.0;
  //values[12] = 14.0;
  //values[18] = 3.0;
  //values[24] = 0.5;
  //values[30] = 22.0;

  Eigen::Map<Eigen::Matrix<float, 6, 6, Eigen::RowMajor>> eigenMatrix(values);
  Eigen6by6Matrix a = eigenMatrix;
  Eigen::Matrix<float, 6, 6> invertedMatrix = inverse(a);

  result[0] = Transformation(invertedMatrix.data());
}

__global__ void test_inverse_kernel(Transformation* result) {

  test_inverse(result);
}

__global__ void forward_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Transformation* result_transformations) {

  int inputs_index = (blockIdx.x * blockDim.x) + threadIdx.x;
  int result_index = inputs_index;
  int offset = blockDim.x * gridDim.x;

  Transformation home_position = Transformation(home_position_values);

  while (inputs_index < num_problems) {
    // Screw* screws, float* thetas, int num_joints, float* home_position_values
    result_transformations[result_index] = forward_kinematics(screw_axes, thetas + (inputs_index * num_joints), home_position, num_joints);

    inputs_index += offset;
    result_index += offset;
  }
}

__global__ void inverse_kinematics_kernel(Screw* screw_axes, float* thetas, float* home_position_values, int num_joints, int num_problems, Eigen6Vector* results) {

  int inputs_index = 0;
  int result_index = 0;

  Transformation home_position = Transformation(home_position_values);
  results[result_index] = inverse_kinematics(screw_axes, thetas + (num_joints * inputs_index), home_position, num_joints);
}


void create_joint_angles(float a, float b, float c, float d, float e, float f, float* output) {
  output[0] = a;
  output[1] = b;
  output[2] = c;
  output[3] = d;
  output[4] = e;
  output[5] = f;
}

Overwriting core.cu


In [146]:
%%writefile test_core.cu
#include "core.hpp"

int main() {

  // Robot info.
  int num_joints = 6;
  int num_configurations = 1; //10550;

  Transformation* result_transformation_host = (Transformation*) malloc(sizeof(Transformation) * num_configurations);
  Transformation* result_transformation_device;
  cudaMalloc((void**)&result_transformation_device, sizeof(Transformation) * num_configurations);

  Eigen6Vector* ik_results_host = (Eigen6Vector*) malloc(sizeof(Eigen6Vector) * num_configurations);
  Eigen6Vector* ik_results_device;
  cudaMalloc((void**)&ik_results_device, sizeof(Eigen6Vector) * num_configurations);

  Screw* screw_axes_host = (Screw*) malloc(sizeof(Screw) * num_joints);
  Screw* screw_axes_device;
  cudaMalloc((void**)&screw_axes_device, sizeof(Screw) * num_joints);

  float* M_host = (float*) malloc(sizeof(float) * Transformation::num_entries_in_matrix);
  float* M_device;
  cudaMalloc((void**)&M_device, sizeof(float) * Transformation::num_entries_in_matrix);

  float* thetas_host = (float*) malloc(sizeof(float) * num_joints * num_configurations);
  float* thetas_device;
  cudaMalloc((void**)&thetas_device, sizeof(float) * num_joints * num_configurations);

  // UR5 Screw axes.
  // See page 148 of Modern Robotics.
  float H1 = 0.089;
  float H2 = 0.095;
  float L1 = 0.425;
  float L2 = 0.392;
  float W1 = 0.109;
  float W2 = 0.082;

  // Define the screw axes.
  screw_axes_host[0] = Screw(Vector3(0., 0., 1.), Vector3(0., 0., 0.));
  screw_axes_host[1] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., 0.));
  screw_axes_host[2] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1));
  screw_axes_host[3] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1 + L2));
  screw_axes_host[4] = Screw(Vector3(0., 0., -1.), Vector3(-W1, L1 + L2, 0.));
  screw_axes_host[5] = Screw(Vector3(0., 1., 0.), Vector3(H2 - H1, 0., L1 + L2));

  // Define the home pose.
  M_host[0] = -1.;
  M_host[1] = 0.;
  M_host[2] = 0.;
  M_host[3] = L1 + L2;

  M_host[4] = 0.;
  M_host[5] = 0.;
  M_host[6] = 1.;
  M_host[7] = W1 + W2;

  M_host[8] = 0.;
  M_host[9] = 1.;
  M_host[10] = 0.;
  M_host[11] = H1 - H2;

  M_host[12] = 0.;
  M_host[13] = 0.;
  M_host[14] = 0.;
  M_host[15] = 1.;

  for (int i = 0; i < num_configurations; i++) {
    create_joint_angles(0., -3.14/2.0, 0., 0., 3.14/2.0, 0., thetas_host + (num_joints * i));
  }

  // Copy the screw axes onto the device.
  cudaMemcpy(screw_axes_device, screw_axes_host, sizeof(Screw) * num_joints, cudaMemcpyHostToDevice);

  // Copy M onto the device.
  cudaMemcpy(M_device, M_host, sizeof(float) * Transformation::num_entries_in_matrix, cudaMemcpyHostToDevice);

  // Copy the joint angles onto the device.
  cudaMemcpy(thetas_device, thetas_host, sizeof(float) * num_joints * num_configurations, cudaMemcpyHostToDevice);

  // Screw* screw_axes, float* thetas_device, float* home_position_values, int num_joints, Transformation* result_transformations
  //forward_kinematics_kernel<<<1, 1>>>(screw_axes_device, thetas_device, M_device, num_joints, num_configurations, result_transformation_device);


  //test_inverse_kernel<<<1, 1>>>(result_transformation_device);

  inverse_kinematics_kernel<<<1, 1>>>(screw_axes_device, thetas_device, M_device, num_joints, num_configurations, ik_results_device);

  int a = cudaMemcpy(result_transformation_host, result_transformation_device, sizeof(Transformation) * num_configurations, cudaMemcpyDeviceToHost);
  std::cout << a << std::endl;

  cudaMemcpy(ik_results_host, ik_results_device, sizeof(Eigen6Vector) * num_configurations, cudaMemcpyDeviceToHost);

  // Display one of the Transformations.
  //result_transformation_host[num_configurations - 1].print();
  std::cout << *ik_results_host << std::endl;

  float* values = (float*) malloc(sizeof(float) * 6 * 6);
  for (int i = 0; i < 6; i++) {
    for (int j = 0; j < 6; j++) {
      values[(i * 6) + j] = 0.0;
      if (i == j) {
        values[(i * 6) + j] = 1.0;
      }
    }
  }

  Eigen::Map<Eigen::Matrix<float, 6, 6, Eigen::RowMajor>> eigenMatrix(values);
  Eigen6by6Matrix aaa = eigenMatrix;
  aaa(0, 3) = 5.0;
  aaa(3, 4) = 30.0;

  std::cout << aaa << std::endl << std::endl;

  Eigen::Matrix<float, 6, 6> invertedMatrix = inverse_cpu(aaa);

  std::cout << invertedMatrix << std::endl;

  // Free the device memory.
  cudaFree(screw_axes_device);
  cudaFree(thetas_device);
  cudaFree(result_transformation_device);
  cudaFree(M_device);

  // Free the host memory.
  free(screw_axes_host);
  free(result_transformation_host);
  free(M_host);

  return 0;
}

Overwriting test_core.cu


In [147]:
%%writefile CPUVector3.hpp
#pragma once

struct CPUVector3 {

    public:
        CPUVector3(float, float, float);
        CPUVector3();
        float x, y, z;
        CPUVector3 operator-(const CPUVector3&) const;
        CPUVector3 operator-(CPUVector3&) const;
        float operator*(CPUVector3&) const;
        float operator*(const CPUVector3&) const;
};

Overwriting CPUVector3.hpp


In [148]:
%%writefile CPUVector3.cpp
#include "CPUVector3.hpp"

CPUVector3::CPUVector3(float x, float y, float z): x(x), y(y), z(z) {
}

CPUVector3::CPUVector3(): x(0.), y(0.), z(0.) {
}

CPUVector3 CPUVector3::operator-(const CPUVector3& right) const {

    return CPUVector3(x - right.x, y - right.y, z - right.z);
}

CPUVector3 CPUVector3::operator-(CPUVector3& right) const {

    return CPUVector3(x - right.x, y - right.y, z - right.z);
}

float CPUVector3::operator*(CPUVector3& right) const {

    return (x * right.x) + (y * right.y) + (z * right.z);
}

float CPUVector3::operator*(const CPUVector3& right) const {

    return (x * right.x) + (y * right.y) + (z * right.z);
}

Overwriting CPUVector3.cpp


In [149]:
%%writefile STLTriangle.hpp
#pragma once

struct STLTriangle {
    float normal[3];
    float v1[3];
    float v2[3];
    float v3[3];
};

Overwriting STLTriangle.hpp


In [150]:
%%writefile triangle.hpp
#pragma once

#include "vectors.hpp"
#include "transformation.hpp"

struct Triangle {

    public:
        __device__ Triangle();
        Triangle(const Vector3&, const Vector3&, const Vector3&);
        Vector3 vertex1;
        Vector3 vertex2;
        Vector3 vertex3;
        __device__ Triangle transform(const Transformation&);
};

Overwriting triangle.hpp


In [151]:
%%writefile triangle_triangle_collision_detector.hpp
#pragma once

#include "triangle.hpp"
#include "transformation.hpp"
#include <iostream>
#include <cmath>


struct TriangleTriangleCollisionDetector {
    // https://fileadmin.cs.lth.se/cs/Personal/Tomas_Akenine-Moller/code/tritri_tam.pdf
    // https://www.youtube.com/watch?v=_W3aVWsMp14

    public:
        TriangleTriangleCollisionDetector();
        __device__ TriangleTriangleCollisionDetector(const Triangle&, const Triangle&);
        __device__ bool check(void);
        const Triangle triangle_1;
        const Triangle triangle_2;
        Vector3 N1, N2;
        float d1, d2;
        float t1_d_v1, t1_d_v2, t1_d_v3;
        float t2_d_v1, t2_d_v2, t2_d_v3;

    private:
        __device__ Vector3 compute_signed_distances(const Triangle&, const Vector3&, float) const;
        __device__ bool do_intervals_intersect(float, float, float, float);
};

Overwriting triangle_triangle_collision_detector.hpp


In [152]:
%%writefile triangle_triangle_collision_detector.cu
#include "triangle_triangle_collision_detector.hpp"

__device__ bool equal(float a, float b, float epsilon = 1e-5) {

    return fabs(a - b) < epsilon;
}

__device__ inline bool is_positive(float a) {
    return a > 0.;
}

__device__ inline bool is_negative(float a) {
    return a < 0.;
}

__device__ bool all_non_zero_same_sign(float a, float b, float c) {

    bool all_not_zero = (!equal(a, 0.)) && (!equal(b, 0.)) && (!equal(c, 0.));
    bool all_positive = is_positive(a) && is_positive(b) && is_positive(c);
    bool all_negative = is_negative(a) && is_negative(b) && is_negative(c);

    return all_not_zero && (all_positive || all_negative);
}

__device__ bool all_zero(float a, float b, float c) {
    return (equal(a, 0.)) && (equal(b, 0.)) && (equal(c, 0.));
}

__device__ bool non_zero_same_sign(float a, float b) {

    bool all_not_zero = (!equal(a, 0.)) && (!equal(b, 0.));
    bool all_positive = is_positive(a) && is_positive(b);
    bool all_negative = is_negative(a) && is_negative(b);

    return all_not_zero && (all_positive || all_negative);
}

__device__ float two_by_two_determinant(float a, float b, float c, float d) {
    /* Where the matrix = [[a, b],
    *                      [c, d]]
    */

   return (a * d) - (b * c);
}

__device__ Vector3 cross_product(const Vector3& left, const Vector3& right) {

    Vector3 v;
    v.x = two_by_two_determinant(left.y, left.z, right.y, right.z);
    v.y = -two_by_two_determinant(left.x, left.z, right.x, right.z);
    v.z = two_by_two_determinant(left.x, left.y, right.x, right.y);

    return v;
}

__device__ float line_segment_intersects_plane(const Vector3& P, const Vector3& Q, const Vector3& N, float D) {
  // Intersection of a line segment and a plane: https://www.youtube.com/watch?v=_W3aVWsMp14
  // To solve the algebraic equation: https://www.wolframalpha.com

   float numerator = -D + (N.x * P.x) + (N.y * P.y) + (N.z * P.z);
   float denominator = (N.x * (P.x - Q.x)) + (N.y * (P.y - Q.y)) + (N.z * (P.z - Q.z));
   return numerator / denominator;
}

__device__ bool TriangleTriangleCollisionDetector::do_intervals_intersect(float a, float b, float c, float d) {

    float lower1, upper1, lower2, upper2;
    lower1 = a <= b ? a : b;
    upper1 = b > a ? b : a;

    lower2 = c <= d ? c : d;
    upper2 = d > c ? d : c;

    if (lower1 >= lower2 && lower1 <= upper2) {
        return true;
    }
    else if (upper1 >= lower2 && upper1 <= upper2) {
        return true;
    }
    else if (lower2 >= lower1 && lower2 <= upper1) {
        return true;
    }
    else if (upper2 >= lower1 && upper2 <= upper1) {
        return true;
    }

    return false;
}

__device__ bool TriangleTriangleCollisionDetector::check(void) {

    if (all_zero(t1_d_v1, t1_d_v2, t1_d_v3) || all_zero(t2_d_v1, t2_d_v2, t2_d_v3)) {
        // The triangles are co-planar. Need to add this check.
        return false;
    }
    else if (all_non_zero_same_sign(t1_d_v1, t1_d_v2, t1_d_v3) || all_non_zero_same_sign(t2_d_v1, t2_d_v2, t2_d_v3)) {
        //Collision ruled out by all the signed distances having the same signs.
        return false;
    }

    Vector3 T1_V1, T1_V2, T1_V3;
    if (non_zero_same_sign(t1_d_v1, t1_d_v2)) {
        T1_V1 = triangle_1.vertex1;
        T1_V2 = triangle_1.vertex2;
        T1_V3 = triangle_1.vertex3;
    }
    else if (non_zero_same_sign(t1_d_v1, t1_d_v3)) {
        T1_V1 = triangle_1.vertex1;
        T1_V2 = triangle_1.vertex3;
        T1_V3 = triangle_1.vertex2;
    }
    else {
        T1_V1 = triangle_1.vertex2;
        T1_V2 = triangle_1.vertex3;
        T1_V3 = triangle_1.vertex1;
    }

    Vector3 T2_V1, T2_V2, T2_V3;
    if (non_zero_same_sign(t2_d_v1, t2_d_v2)) {
        T2_V1 = triangle_2.vertex1;
        T2_V2 = triangle_2.vertex2;
        T2_V3 = triangle_2.vertex3;
    }
    else if (non_zero_same_sign(t2_d_v1, t2_d_v3)) {
        T2_V1 = triangle_2.vertex1;
        T2_V2 = triangle_2.vertex3;
        T2_V3 = triangle_2.vertex2;
    }
    else {
        T2_V1 = triangle_2.vertex2;
        T2_V2 = triangle_2.vertex3;
        T2_V3 = triangle_2.vertex1;
    }

    float t1_a = line_segment_intersects_plane(T1_V1, T1_V3, N2, d2);
    float t1_b = line_segment_intersects_plane(T1_V2, T1_V3, N2, d2);

    float t2_a = line_segment_intersects_plane(T2_V1, T2_V3, N1, d1);
    float t2_b = line_segment_intersects_plane(T2_V2, T2_V3, N1, d1);

    return do_intervals_intersect(t1_a, t1_b, t2_a, t2_b);
}

__device__ TriangleTriangleCollisionDetector::TriangleTriangleCollisionDetector(const Triangle& T1, const Triangle& T2): triangle_1(T1), triangle_2(T2) {

    N1 = cross_product((T1.vertex2 - T1.vertex1), (T1.vertex3 - T1.vertex1));
    N2 = cross_product((T2.vertex2 - T2.vertex1), (T2.vertex3 - T2.vertex1));

    d1 = -(N1 * T1.vertex1);
    d2 = -(N2 * T2.vertex1);

    Vector3 T0_vertex_distances = compute_signed_distances(T1, N2, d2);
    Vector3 T1_vertex_distances = compute_signed_distances(T2, N1, d1);

    t1_d_v1 = T0_vertex_distances.x;
    t1_d_v2 = T0_vertex_distances.y;
    t1_d_v3 = T0_vertex_distances.z;

    t2_d_v1 = T1_vertex_distances.x;
    t2_d_v2 = T1_vertex_distances.y;
    t2_d_v3 = T1_vertex_distances.z;
}

__device__ Vector3 TriangleTriangleCollisionDetector::compute_signed_distances(const Triangle& T, const Vector3& N, float d) const {

    Vector3 v;
    v.x = (N * T.vertex1) + d;
    v.y = (N * T.vertex2) + d;
    v.z = (N * T.vertex3) + d;

    return v;
}

Overwriting triangle_triangle_collision_detector.cu


In [153]:
%%writefile triangle.hpp
#pragma once

#include "vectors.hpp"
#include "transformation.hpp"


struct Triangle {

    public:
        __device__ Triangle();
        Triangle(const Vector3, const Vector3, const Vector3);
        Vector3 vertex1;
        Vector3 vertex2;
        Vector3 vertex3;
        __device__ Triangle transform(const Transformation&);
};

Overwriting triangle.hpp


In [154]:
%%writefile triangle.cu
#include "triangle.hpp"
#include <iostream>

__device__ Triangle::Triangle() {

  this->vertex1 = Vector3();
  this->vertex2 = Vector3();
  this->vertex3 = Vector3();
}

Triangle::Triangle(const Vector3 vertex1, const Vector3 vertex2, const Vector3 vertex3): vertex1(vertex1), vertex2(vertex2), vertex3(vertex3) {
}

__device__ Triangle Triangle::transform(const Transformation& transformation) {

  Triangle transformed_triangle = Triangle();

  transformed_triangle.vertex1 = transformation * vertex1;
  transformed_triangle.vertex2 = transformation * vertex2;
  transformed_triangle.vertex3 = transformation * vertex3;

  return transformed_triangle;
}

Overwriting triangle.cu


In [155]:
%%writefile mesh.hpp
#pragma once

#include "read_stl.hpp"
#include "triangle.hpp"
#include "transformation.hpp"
#include <iostream>
#include <string>
#include <vector>

class Mesh {

    public:
        Mesh();
        void load_from_file(std::string);
        void print_triangles(void);
        Triangle* triangles;
        int num_triangles;
        std::string file_name;
};

Overwriting mesh.hpp


In [156]:
%%writefile read_stl.hpp
#pragma once

#include "STLTriangle.hpp"
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <stdexcept>
#include <cmath>


std::vector<STLTriangle> parseBinarySTL(const std::string& filename);

Overwriting read_stl.hpp


In [157]:
%%writefile read_stl.cu
#include "read_stl.hpp"

std::vector<STLTriangle> parseBinarySTL(const std::string& filename) {

    std::vector<STLTriangle> triangles;
    std::ifstream file(filename, std::ios::binary);

    if (!file.is_open()) {
        std::cerr << "Error opening file: " << filename << std::endl;
        return triangles;
    }

    // Read 80-byte header (ignore)
    file.seekg(80, std::ios::beg);

    // Read number of triangles (4-byte unsigned int)
    uint32_t numTriangles;
    file.read(reinterpret_cast<char*>(&numTriangles), sizeof(numTriangles));

    // Read each triangle
    for (uint32_t i = 0; i < numTriangles; ++i) {
        STLTriangle triangle;
        // Read normal vector (3 floats, 12 bytes)
        file.read(reinterpret_cast<char*>(&triangle.normal), sizeof(triangle.normal));
        // Read vertices (3 * 3 floats, 36 bytes)
        file.read(reinterpret_cast<char*>(&triangle.v1), sizeof(triangle.v1));
        file.read(reinterpret_cast<char*>(&triangle.v2), sizeof(triangle.v2));
        file.read(reinterpret_cast<char*>(&triangle.v3), sizeof(triangle.v3));
        // Read attribute count (2-byte unsigned int, ignored)
        file.seekg(2, std::ios::cur);
        triangles.push_back(triangle);
}

    return triangles;
}

Overwriting read_stl.cu


In [158]:
%%writefile mesh.cu
#include "mesh.hpp"

Mesh::Mesh(): triangles(nullptr), file_name(""), num_triangles(0) {
}

void Mesh::print_triangles(void) {

    if (!triangles) {
        std::cout << "There are no triangles in this mesh!" << std::endl;
        return;
    }

    std::cout << "Number of triangles: " << num_triangles << std::endl;
    for (int i = 0; i < num_triangles; i++) {
        auto& triangle = triangles[i];
        std::cout << "Vertex 1: " << triangle.vertex1.x << " " << triangle.vertex1.y << " " << triangle.vertex1.z << std::endl;
        std::cout << "Vertex 2: " << triangle.vertex2.x << " " << triangle.vertex2.y << " " << triangle.vertex2.z << std::endl;
        std::cout << "Vertex 3: " << triangle.vertex3.x << " " << triangle.vertex3.y << " " << triangle.vertex3.z << std::endl;
    }
}

void Mesh::load_from_file(std::string file_name) {

    std::vector<STLTriangle> read_triangles = parseBinarySTL(file_name);
    this->num_triangles = read_triangles.size();
    triangles = (Triangle*) malloc(sizeof(Triangle) * num_triangles);
    for (int i = 0; i < read_triangles.size(); i++) {
        auto v1 = Vector3(read_triangles[i].v1[0], read_triangles[i].v1[1], read_triangles[i].v1[2]);
        auto v2 = Vector3(read_triangles[i].v2[0], read_triangles[i].v2[1], read_triangles[i].v2[2]);
        auto v3 = Vector3(read_triangles[i].v3[0], read_triangles[i].v3[1], read_triangles[i].v3[2]);
        triangles[i] = Triangle(v1, v2, v3);
    }
}

Overwriting mesh.cu


In [159]:
%%writefile triangle_triangle_collision_kernels.hpp
#include "triangle.hpp"
#include "mesh.hpp"
#include "triangle_triangle_collision_detector.hpp"

__global__ void collisions_kernel(Triangle* triangles, int num_triangles, bool* results);

Overwriting triangle_triangle_collision_kernels.hpp


In [160]:
%%writefile triangle_triangle_collision_kernels.cu
#include "triangle_triangle_collision_kernels.hpp"

__global__ void collisions_kernel(Triangle* triangles, int num_triangles, bool* results) {

  int start_triangle_1_index = (blockIdx.x * blockDim.x) + threadIdx.x; // The row in the grid.
  int start_triangle_2_index = (blockIdx.y * blockDim.y) + threadIdx.y; // The column in the grid.

  int x_offset = gridDim.x * blockDim.x; // The total number of threads along the x-axis.
  int y_offset = gridDim.y * blockDim.y; // The total number of threads along the y-axis.

  int triangle_1_index = start_triangle_1_index;
  int triangle_2_index = start_triangle_2_index;

  while (triangle_1_index < num_triangles) {

    while (triangle_2_index < num_triangles) {

      Triangle T1 = triangles[triangle_1_index];
      Triangle T2 = triangles[triangle_2_index];
      int result_index = triangle_1_index + (triangle_2_index * num_triangles);

      TriangleTriangleCollisionDetector detector = TriangleTriangleCollisionDetector(T1, T2);
      results[result_index] = detector.check();

      triangle_2_index += y_offset;
    }

    triangle_1_index += x_offset;
    triangle_2_index += start_triangle_2_index;
  }
}

Overwriting triangle_triangle_collision_kernels.cu


In [161]:
%%writefile test_triangle_triangle_collisions.cu
#include "triangle.hpp"
#include "mesh.hpp"
#include "triangle_triangle_collision_detector.hpp"
#include "triangle_triangle_collision_kernels.cu"
#include <cassert>

int main() {

    // Allocate space for the two triangles.
    int num_triangles = 3;
    int num_collision_checks = num_triangles * num_triangles;
    Triangle* triangles_host = (Triangle*) malloc(sizeof(Triangle) * num_triangles);
    Triangle* triangles_device;
    cudaMalloc((void**)&triangles_device, sizeof(Triangle) * num_triangles);

    // Define the triangles.
    triangles_host[0] = Triangle(Vector3(0., -1., 0.), Vector3(0., 1., 0.), Vector3(0., 0., 1.));
    triangles_host[1] = Triangle(Vector3(-10., -1., 0.), Vector3(-1000., -30., 0.), Vector3(-200., -20., 1.));
    triangles_host[2] = Triangle(Vector3(-1., 0., 0.), Vector3(1., 0., 0.), Vector3(0., 0., 0.5));

    // Define the expected results.
    bool* expected_collisions = (bool*) calloc(num_collision_checks, sizeof(bool));
    expected_collisions[2] = true;
    expected_collisions[6] = true;

    // Allocate space for the result.
    bool* results_host = (bool*) malloc(sizeof(bool) * num_collision_checks);
    bool* results_device;
    cudaMalloc((void**)&results_device, sizeof(bool) * num_collision_checks);

    // Copy the triangles onto the device.
    int b = cudaMemcpy(triangles_device, triangles_host, sizeof(Triangle) * num_triangles, cudaMemcpyHostToDevice);
    std::cout << b << std::endl;

    // Call the collision checking kernel.
    dim3 threadsPerBlock(num_triangles, num_triangles);
    collisions_kernel<<<1, threadsPerBlock>>>(triangles_device, num_triangles, results_device);

    // Copy the results from the GPU to the CPU.
    int a = cudaMemcpy(results_host, results_device, sizeof(bool) * num_collision_checks, cudaMemcpyDeviceToHost);
    std::cout << a << std::endl;

    for (int i = 0; i < num_collision_checks; i++) {
      assert(results_host[i] == expected_collisions[i]);
    }

    // Free the GPU memory.
    cudaFree(triangles_device);
    cudaFree(results_device);

    // Free the CPU memory.
    free(triangles_host);
    free(results_host);

    return 0;
}

Overwriting test_triangle_triangle_collisions.cu


In [162]:
%%writefile collision_kernels.hpp
#include "mesh.hpp"
#include "transformation.hpp"
#include "triangle.hpp"
#include "screw.hpp"
#include "triangle_triangle_collision_detector.hpp"
#include "core.hpp"

__global__ void is_in_self_collision_kernel(Screw* screw_axes, float* thetas, Triangle* mesh_triangles, int* num_triangles_in_meshes, int num_joints, int num_problems, bool* results);


__device__ bool collision_check(Triangle*, Triangle*, int, int);
__device__ bool collision_check(Mesh& mesh1, Mesh& mesh2, Transformation& T1, Transformation& T2);

Overwriting collision_kernels.hpp


In [163]:
%%writefile collision_kernels.cu
#include "collision_kernels.hpp"

__device__ int sum(int* nums, int n) {

  int total = 0;
  for (int i = 0; i < n; i++) {
    total += nums[i];
  }

  return total;
}

__device__ bool collision_check(Triangle* triangles1, Triangle* triangles2, int num_triangles1, int num_triangles2) {

  for (int i = 0; i < num_triangles1; i++) {

    Triangle& T1 = triangles1[i];

    for (int j = 0; j < num_triangles2; j++) {

      Triangle& T2 = triangles2[j];

      TriangleTriangleCollisionDetector detector = TriangleTriangleCollisionDetector(T1, T2);

      if (detector.check()) {
        return true;
      }
    }
  }

  return false;
}

__device__ bool collision_check(Triangle* triangles1, Triangle* triangles2, int num_triangles_1, int num_triangles_2, Transformation& T1, Transformation& T2) {

  // Transform the points of each mesh.
  Triangle* transformed_triangles_1 = (Triangle*) malloc(sizeof(Triangle) * num_triangles_1);
  Triangle* transformed_triangles_2 = (Triangle*) malloc(sizeof(Triangle) * num_triangles_2);

  for (int i = 0; i < num_triangles_1; i++) {
    transformed_triangles_1[i] = triangles1[i].transform(T1);
  }

  for (int i = 0; i < num_triangles_2; i++) {
    transformed_triangles_2[i] = triangles2[i].transform(T2);
  }

  bool result = collision_check(transformed_triangles_1, transformed_triangles_2, num_triangles_1, num_triangles_2);

  free(transformed_triangles_1);
  free(transformed_triangles_2);

  return result;
}

__global__ void is_in_self_collision_kernel(Screw* screw_axes, float* thetas, Triangle* mesh_triangles, int* num_triangles_in_meshes, int num_joints, int num_problems, bool* results) {

  int inputs_index = 0;
  int result_index = inputs_index;
  int offset = 0;

  results[result_index] = false;

  Transformation* joint_to_joint_transformations = forward_kinematics_joint_transformations(screw_axes, thetas, num_joints);

  int num_triangles_1;
  int num_triangles_2;

  for (int i = 0; i < num_joints; i++) {

    Triangle* triangles1 = mesh_triangles + sum(num_triangles_in_meshes, i);
    num_triangles_1 = num_triangles_in_meshes[i];

    Transformation base_to_joint_of_mesh_1 = multiply(joint_to_joint_transformations, i);

    for (int j = i + 1; j < num_joints; j++) {

      Transformation base_to_joint_of_mesh_2 = multiply(joint_to_joint_transformations, j);

      Triangle* triangles2 = mesh_triangles + sum(num_triangles_in_meshes, j);
      num_triangles_2 = num_triangles_in_meshes[j];

      if (collision_check(triangles1, triangles2, num_triangles_1, num_triangles_2, base_to_joint_of_mesh_1, base_to_joint_of_mesh_2)) {
        results[result_index] = true;
      }
    }
  }

  free(joint_to_joint_transformations);
}

Overwriting collision_kernels.cu


In [180]:
%%writefile test_is_in_self_collision_kernel.cu
#include "collision_kernels.hpp"
#include "transformation.hpp"
#include "screw.hpp"
#include "core.hpp"
#include <iostream>
#include <cassert>

int main() {

  // base -> shoulder -> upperarm -> forearm -> wrist1 -> wrist2 -> wrist3

  // Load the UR5 collision meshes.
  Mesh base = Mesh();
  base.load_from_file("./meshes/base.stl");
  Triangle* base_triangles = base.triangles;
  int num_triangles_in_base = base.num_triangles;

  Mesh shoulder = Mesh();
  shoulder.load_from_file("./meshes/shoulder.stl");
  Triangle* shoulder_triangles = shoulder.triangles;
  int num_triangles_in_shoulder = shoulder.num_triangles;

  Mesh upperarm = Mesh();
  upperarm.load_from_file("./meshes/upperarm.stl");
  Triangle* upperarm_triangles = upperarm.triangles;
  int num_triangles_in_upperarm = upperarm.num_triangles;

  Mesh forearm = Mesh();
  forearm.load_from_file("./meshes/forearm.stl");
  Triangle* forearm_triangles = forearm.triangles;
  int num_triangles_in_forearm = forearm.num_triangles;

  Mesh wrist1 = Mesh();
  wrist1.load_from_file("./meshes/wrist1.stl");
  Triangle* wrist1_triangles = wrist1.triangles;
  int num_triangles_in_wrist1 = wrist1.num_triangles;

  Mesh wrist2 = Mesh();
  wrist2.load_from_file("./meshes/wrist2.stl");
  Triangle* wrist2_triangles = wrist2.triangles;
  int num_triangles_in_wrist2 = wrist2.num_triangles;

  Mesh wrist3 = Mesh();
  wrist3.load_from_file("./meshes/wrist3.stl");
  Triangle* wrist3_triangles = wrist3.triangles;
  int num_triangles_in_wrist3 = wrist3.num_triangles;

  //Triangle** triangles = (Triangle**) malloc(sizeof(Triangle*) * num_joints);
  //float* nums_of_triangles = (float*) malloc(sizeof(float) * num_joints);

  // Define constants.
  int num_joints = 6;
  int num_problems = 1;
  int* num_triangles_in_meshes_host = (int*) malloc(sizeof(int) * num_joints);

  // Enter the mesh information.
  num_triangles_in_meshes_host[0] = num_triangles_in_shoulder;
  num_triangles_in_meshes_host[1] = num_triangles_in_upperarm;
  num_triangles_in_meshes_host[2] = num_triangles_in_forearm;
  num_triangles_in_meshes_host[3] = num_triangles_in_wrist1;
  num_triangles_in_meshes_host[4] = num_triangles_in_wrist2;
  num_triangles_in_meshes_host[5] = num_triangles_in_wrist3;

  int num_triangles_in_all_meshes = 0;
  for (int i = 0; i < num_joints; i++) {
    num_triangles_in_all_meshes += num_triangles_in_meshes_host[i];
  }

  int* num_triangles_in_meshes_device;
  cudaMalloc((void**)&num_triangles_in_meshes_device, sizeof(int) * num_joints);

  Screw* screw_axes_host = (Screw*) malloc(sizeof(Screw) * num_joints);
  Screw* screw_axes_device;
  cudaMalloc((void**)&screw_axes_device, sizeof(Screw) * num_joints);

  float* thetas_host = (float*) malloc(sizeof(float) * num_joints);
  float* thetas_device;
  cudaMalloc((void**)&thetas_device, sizeof(float) * num_joints);

  Triangle* mesh_triangles_host = (Triangle*) malloc(sizeof(Triangle) * num_triangles_in_all_meshes);
  Triangle* mesh_triangles_device;
  cudaMalloc((void**)&mesh_triangles_device, sizeof(Triangle) * num_triangles_in_all_meshes);

  bool* results_host = (bool*) malloc(sizeof(bool) * num_problems);
  bool* results_device;
  cudaMalloc((void**)&results_device, sizeof(bool) * num_problems);

  // Define the expected results.
  bool* expected = (bool*) calloc(num_problems, sizeof(bool));

  // UR5 Screw axes.
  // See page 148 of Modern Robotics.
  float H1 = 0.089;
  float H2 = 0.095;
  float L1 = 0.425;
  float L2 = 0.392;
  float W1 = 0.109;
  float W2 = 0.082;

  // Screw axes for a UR5 robot.
  screw_axes_host[0] = Screw(Vector3(0., 0., 1.), Vector3(0., 0., 0.));
  screw_axes_host[1] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., 0.));
  screw_axes_host[2] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1));
  screw_axes_host[3] = Screw(Vector3(0., 1., 0.), Vector3(-H1, 0., L1 + L2));
  screw_axes_host[4] = Screw(Vector3(0., 0., -1.), Vector3(-W1, L1 + L2, 0.));
  screw_axes_host[5] = Screw(Vector3(0., 1., 0.), Vector3(H2 - H1, 0., L1 + L2));

  thetas_host[0] = 0.;
  thetas_host[1] = 0.;
  thetas_host[2] = 0.;
  thetas_host[3] = 0.;
  thetas_host[4] = 0.;
  thetas_host[5] = 0.;

  num_triangles_in_meshes_host[0] = num_triangles_in_shoulder;
  num_triangles_in_meshes_host[1] = num_triangles_in_upperarm;
  num_triangles_in_meshes_host[2] = num_triangles_in_forearm;
  num_triangles_in_meshes_host[3] = num_triangles_in_wrist1;
  num_triangles_in_meshes_host[4] = num_triangles_in_wrist2;
  num_triangles_in_meshes_host[5] = num_triangles_in_wrist3;

  Triangle* tmp = mesh_triangles_host;

  memcpy(tmp, shoulder_triangles, num_triangles_in_shoulder);
  tmp += num_triangles_in_shoulder;
  memcpy(tmp, upperarm_triangles, num_triangles_in_upperarm);
  tmp += num_triangles_in_upperarm;
  memcpy(tmp, forearm_triangles, num_triangles_in_forearm);
  tmp += num_triangles_in_forearm;
  memcpy(tmp, wrist1_triangles, num_triangles_in_wrist1);
  tmp += num_triangles_in_wrist1;
  memcpy(tmp, wrist2_triangles, num_triangles_in_wrist2);
  tmp += num_triangles_in_wrist2;
  memcpy(tmp, wrist3_triangles, num_triangles_in_wrist3);
  tmp += num_triangles_in_wrist3;

  // Copy the screw axes onto the device.
  cudaMemcpy(screw_axes_device, screw_axes_host, sizeof(Screw) * num_joints, cudaMemcpyHostToDevice);

  // Copy the joint angles onto the device.
  cudaMemcpy(thetas_device, thetas_host, sizeof(float) * num_joints * num_problems, cudaMemcpyHostToDevice);

  // Copy these items onto the device.
  cudaMemcpy(mesh_triangles_device, mesh_triangles_host, sizeof(Triangle) * num_triangles_in_all_meshes, cudaMemcpyHostToDevice);

  cudaMemcpy(num_triangles_in_meshes_device, num_triangles_in_meshes_host, sizeof(float) * num_joints, cudaMemcpyHostToDevice);

  is_in_self_collision_kernel<<<1, 1>>>(screw_axes_device, thetas_device, mesh_triangles_device, num_triangles_in_meshes_device, num_joints, num_problems, results_device);

  cudaMemcpy(results_host, results_device, sizeof(bool) * num_problems, cudaMemcpyDeviceToHost);

  if (results_host[0]) {
    std::cout << "Collision" << std::endl;
  }
  else {
    std::cout << "No Collision" << std::endl;
  }

  return 0;
}

Overwriting test_is_in_self_collision_kernel.cu


In [165]:
%%script bash
nvcc -diag-suppress 20012 test_triangle_triangle_collisions.cu vectors.cu triangle.cu read_stl.cu mesh.cu triangle_triangle_collision_detector.cu -rdc=true -o test_triangle_triangle_collisions

In [166]:
%%script bash
./test_triangle_triangle_collisions

0
0


In [181]:
%%script bash
nvcc -diag-suppress 20012 test_is_in_self_collision_kernel.cu core.cu vectors.cu transformation.cu screw.cu triangle.cu read_stl.cu mesh.cu triangle_triangle_collision_detector.cu collision_kernels.cu -rdc=true -o test_is_in_self_collision_kernel

test_is_in_self_collision_kernel.cu(15): warning #177-D: variable "base_triangles" was declared but never referenced
    Triangle* base_triangles = base.triangles;
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

test_is_in_self_collision_kernel.cu(16): warning #177-D: variable "num_triangles_in_base" was declared but never referenced
    int num_triangles_in_base = base.num_triangles;
        ^

test_is_in_self_collision_kernel.cu(98): warning #177-D: variable "W2" was declared but never referenced
    float W2 = 0.082;
          ^

collision_kernels.cu(60): warning #177-D: variable "offset" was declared but never referenced
    int offset = 0;
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [183]:
%%script bash
./test_is_in_self_collision_kernel

Collision


In [ ]:
%%script bash
ls -lash ./meshes

In [ ]:
%%script bash
./forward_kinematics